# Phuc Swarms Orchestration (Secret Sauce)

**Mission:** Demonstrate a portable orchestration pattern (DREAM → FORECAST → DECIDE → ACT → VERIFY) with fail-closed gates and context isolation.

**Auth:** 65537 (project tag)


```mermaid
flowchart TD
  A[DREAM: Scout\nSCOUT_REPORT.json] --> B[FORECAST: Grace\nFORECAST_MEMO.json]
  B --> C[DECIDE: Judge\nDECISION_RECORD.json]
  C --> D[ACT: Solver\nPATCH_PROPOSAL.diff]
  D --> E[VERIFY: Skeptic\nSKEPTIC_VERDICT.json]
  E -->|REJECTED| D
  E -->|APPROVED| F[EXIT_PASS]

  classDef phase fill:#0b1b2b,stroke:#9cc3ff,color:#e6f0ff;
  class A,B,C,D,E phase;
```

Notes:
- Runs fully offline by default (`STILLWATER_DEMO=1`).
- For real LLM calls, set `STILLWATER_DEMO=0` and provide `STILLWATER_WRAPPER_URL`.


## Setup: Dependencies and Configuration

Default (portable):
- Python 3.10+
- No external services required (offline demo mode)
- Default `WORK_DIR` uses your OS temp directory

Optional (LLM-backed):
- A local wrapper (or any compatible endpoint)
- Set `STILLWATER_DEMO=0` and `STILLWATER_WRAPPER_URL=http://localhost:8080/api/generate`

Optional (real SWE-bench runs):
- SWE-bench data available locally (path configured via `STILLWATER_SWE_BENCH_DATA`)


In [1]:
import json
import subprocess
import tempfile
import shutil
import re
import os
import shlex
import sys
import urllib.request
import urllib.error
from pathlib import Path
from typing import Optional, Dict, Any

# Configuration (portable defaults)
DATA_DIR = Path(os.environ.get(
    'STILLWATER_SWE_BENCH_DATA',
    str(Path.home() / 'Downloads/benchmarks/SWE-bench-official'),
))

DEFAULT_WORK_DIR = Path(tempfile.gettempdir()) / 'phuc-swarms-demo'
WORK_DIR = Path(os.environ.get('STILLWATER_WORK_DIR', str(DEFAULT_WORK_DIR)))
WORK_DIR.mkdir(exist_ok=True, parents=True)

# Notebook runs in offline demo mode by default.
DEMO_MODE = os.environ.get('STILLWATER_DEMO', '1') == '1'
MODE = 'DEMO' if DEMO_MODE else 'REAL'
WRAPPER_URL = os.environ.get('STILLWATER_WRAPPER_URL', 'http://localhost:8080/api/generate')

# Prime Coder policy: declare rung target before claiming PASS.
# Default 641 = local correctness claim (RED→GREEN + no regressions in existing tests).
VERIFICATION_RUNG_TARGET = int(os.environ.get('STILLWATER_VERIFICATION_RUNG_TARGET', '641'))


def _pretty_path(p: Path) -> str:
    # Redact $HOME to keep committed notebook outputs portable.
    try:
        home = str(Path.home())
        return str(p).replace(home, '$HOME')
    except Exception:
        return str(p)


print(f"✓ Mode: {MODE}")
print(f"✓ Verification rung target: {VERIFICATION_RUNG_TARGET}")
print(f"✓ Working directory: {_pretty_path(WORK_DIR)}")
print(f"✓ Data directory: {_pretty_path(DATA_DIR)}")
print(f"✓ Data available: {DATA_DIR.exists()}")
print(f"✓ Wrapper URL: {WRAPPER_URL}")


def _extract_json_dict(text: str) -> Optional[Dict[str, Any]]:
    # Best-effort extraction of the first JSON object from arbitrary text.
    # Intentionally avoids fragile regex-based JSON parsing.
    if not text:
        return None

    try:
        obj = json.loads(text)
        if isinstance(obj, dict):
            return obj
    except Exception:
        pass

    decoder = json.JSONDecoder()
    for i, ch in enumerate(text):
        if ch != '{':
            continue
        try:
            obj, _end = decoder.raw_decode(text[i:])
        except Exception:
            continue
        if isinstance(obj, dict):
            return obj

    return None


def _call_wrapper(payload: Dict[str, Any]) -> Optional[str]:
    # Best-effort wrapper call using stdlib (portable; no curl dependency).
    try:
        data = json.dumps(payload).encode('utf-8')
        req = urllib.request.Request(
            WRAPPER_URL,
            data=data,
            headers={'Content-Type': 'application/json'},
            method='POST',
        )
        with urllib.request.urlopen(req, timeout=30) as resp:
            body = resp.read().decode('utf-8', errors='replace')
        obj = json.loads(body)
        if not isinstance(obj, dict):
            return None
        return obj.get('response', '')
    except Exception:
        return None


def _demo_scout(problem: str, error: str, source: str) -> Dict[str, Any]:
    # Minimal deterministic extractor for this notebook's synthetic tests.
    failing = []
    m = re.search(r'^FAILED\s+([^\n]+)$', error, re.MULTILINE)
    if m:
        failing = [m.group(1).strip()]

    suspect = []
    if 'calculator.py' in source:
        suspect.append('calculator.py')
    if failing and 'tests/' in failing[0]:
        suspect.append(failing[0].split('::')[0])
    if not suspect:
        suspect = ['(unknown)']

    return {
        'task_summary': 'Fix bug based on failing test and traceback',
        'repro_command': 'pytest -xvs',
        'failing_tests': failing or ['(unknown)'],
        'suspect_files': suspect,
        'acceptance_criteria': ['failing test passes', 'no regressions'],
    }


def _demo_grace() -> Dict[str, Any]:
    return {
        'top_failure_modes_ranked': [
            {'mode': 'Patch changes behavior for edge cases', 'risk_level': 'HIGH'},
            {'mode': 'Patch breaks type/None handling', 'risk_level': 'MED'},
            {'mode': 'Patch introduces performance regression', 'risk_level': 'LOW'},
        ],
        'edge_cases_to_test': ['empty list', 'all negative', 'mixed ints/floats'],
        'compatibility_risks': ['behavior change for callers relying on old bug'],
        'stop_rules': ['any existing tests fail', 'patch not minimal'],
    }


def _demo_diff() -> str:
    # NOTE: Blank lines inside hunks must be prefixed with a single space.
    lines = [
        '--- a/calculator.py',
        '+++ b/calculator.py',
        '@@ -1,8 +1,7 @@',
        ' def calculate_total(numbers):',
        '     # Calculate sum of all numbers in the list.',
        '     total = 0',
        '     for num in numbers:',
        '-        if num > 0:  # BUG: ignores negative numbers',
        '-            total += num',
        '+        total += num',
        '     return total',
        ' ',
    ]
    return "\n".join(lines) + "\n"


print('✓ Notebook helpers defined')


✓ Mode: DEMO
✓ Verification rung target: 641
✓ Working directory: /tmp/phuc-swarms-demo
✓ Data directory: $HOME/Downloads/benchmarks/SWE-bench-official
✓ Data available: True
✓ Wrapper URL: http://localhost:8080/api/generate
✓ Notebook helpers defined


## Phase 1: DREAM - Scout Agent (Problem Analysis)

### What Scout Does
Scout (Linus Torvalds persona) analyzes a real SWE-bench instance and answers:
1. **What's the bug?** (one sentence summary)
2. **How to reproduce it?** (exact pytest command)
3. **Which tests fail?** (specific test names)
4. **What files to fix?** (ranked by priority)
5. **How do we know it's fixed?** (acceptance criteria)

### The Secret Sauce: Fail-Closed Prompting
- **❌ Don't do:** "If you can't analyze, output NEED_INFO" → Forces Haiku to give up
- **✅ Do:** "YOU MUST analyze using context provided" → Forces Haiku to think harder

### Key Prompting Rules
1. **No escape hatches** - Don't give Haiku a way out
2. **Full context** - Provide complete problem, error, and source
3. **Directive tone** - "YOU MUST", "CRITICAL", "REQUIRED"
4. **Inference rules** - Tell Haiku HOW to infer missing pieces
5. **Explicit format** - Show exact JSON schema expected

In [2]:
# Phase 1: DREAM - Scout Agent

def scout_analyze(instance_id: str, problem: str, error: str, source: str) -> Dict[str, Any]:
    # Scout emits SCOUT_REPORT.json.

    if DEMO_MODE:
        out = _demo_scout(problem=problem, error=error, source=source)
        out['mode_used'] = 'DEMO'
        return out

    system = '''AUTHORITY: 65537 (Phuc Forecast + Prime Coder + Phuc Context)

PERSONA: Linus Torvalds (Linux kernel debugging master)
ROLE: DREAM phase - Define what "fixed" means, locate suspects, minimal repro

YOU MUST OUTPUT VALID JSON. NO QUESTIONS, NO ESCAPE HATCHES.

REQUIRED JSON SCHEMA:
{
  "task_summary": "one sentence: what's broken?",
  "repro_command": "exact pytest command to reproduce (parse from error output if needed)",
  "failing_tests": ["list of test names from error output"],
  "suspect_files": ["files mentioned in problem or error, highest priority first"],
  "acceptance_criteria": ["test passes without failure", "no regressions"]
}

OUTPUT ONLY JSON.
'''

    prompt = f'''REAL SWE-BENCH INSTANCE:

PROBLEM STATEMENT:
{problem}

PYTEST ERROR OUTPUT:
{error}

SOURCE CODE CONTEXT:
{source}

SCOUT TASK: Emit valid JSON:
'''

    payload = {
        'system': system,
        'prompt': prompt,
        'model': 'haiku',
        'stream': False,
    }

    response = _call_wrapper(payload)
    scout_json = _extract_json_dict(response or '')
    if isinstance(scout_json, dict):
        required = [
            'task_summary',
            'repro_command',
            'failing_tests',
            'suspect_files',
            'acceptance_criteria',
        ]
        if all(k in scout_json for k in required):
            scout_json.setdefault('mode_used', 'REAL')
            return scout_json

    # Fail-closed: schema-valid output
    out = _demo_scout(problem=problem, error=error, source=source)
    out['mode_used'] = 'DEMO_FALLBACK'
    return out


print('✓ Scout agent defined')
print('  Phase: DREAM')
print('  Output: SCOUT_REPORT.json')


✓ Scout agent defined
  Phase: DREAM
  Output: SCOUT_REPORT.json


## Phase 2: FORECAST - Grace Agent (Failure Analysis)

### What Grace Does
Grace (Grace Hopper persona) performs a premortem: "How will this patch fail?"
1. **Top failure modes** - Ranked by severity (HIGH/MED/LOW)
2. **Edge cases** - What specific scenarios might break?
3. **Compatibility risks** - Python versions, platforms, backwards-compat?
4. **Stop rules** - When should we reject the patch?

### Why Grace Works
- Gets fresh context (Scout report + problem + error)
- Doesn't see prior reasoning (anti-rot)
- Forced to be concrete (not "might have issues" but specific failure modes)
- Already working well in tests ✅

In [3]:
# Phase 2: FORECAST - Grace Agent

def grace_forecast(scout_report: Dict[str, Any], problem: str, error: str) -> Dict[str, Any]:
    # Grace emits FORECAST_MEMO.json.

    if DEMO_MODE:
        out = _demo_grace()
        out['mode_used'] = 'DEMO'
        return out

    system = '''AUTHORITY: 65537 (Phuc Forecast + Prime Coder)

PERSONA: Grace Hopper
ROLE: FORECAST phase - Premortem

OUTPUT ONLY JSON.
'''

    prompt = f'''FRESH CONTEXT (Anti-Rot):

SCOUT FOUND:
{json.dumps(scout_report, indent=2)}

PROBLEM:
{problem[:400]}

ERROR:
{error[:500]}

OUTPUT ONLY JSON:
'''

    payload = {
        'system': system,
        'prompt': prompt,
        'model': 'haiku',
        'stream': False,
    }

    response = _call_wrapper(payload)
    grace_json = _extract_json_dict(response or '')
    if isinstance(grace_json, dict):
        required = [
            'top_failure_modes_ranked',
            'edge_cases_to_test',
            'compatibility_risks',
            'stop_rules',
        ]
        if all(k in grace_json for k in required):
            grace_json.setdefault('mode_used', 'REAL')
            return grace_json

    out = _demo_grace()
    out['mode_used'] = 'DEMO_FALLBACK'
    return out


print('✓ Grace agent defined')
print('  Phase: FORECAST')
print('  Output: FORECAST_MEMO.json')


✓ Grace agent defined
  Phase: FORECAST
  Output: FORECAST_MEMO.json


## Phase 3: DECIDE - Judge Agent (Decision Lock)

### What Judge Does
Judge (strict reviewer persona) makes the process binding:
1. Locks scope (what files are allowed to change)
2. Selects an approach (with a rationale)
3. Declares verification strength (rung target) and stop rules

This prevents the common failure mode: Solver does something clever but unverifiable.


In [4]:
# Phase 3: DECIDE - Judge Agent

def judge_decide(
    scout_report: Dict[str, Any],
    forecast_memo: Dict[str, Any],
    verification_rung_target: int = VERIFICATION_RUNG_TARGET,
) -> Dict[str, Any]:
    # Judge emits DECISION_RECORD.json.

    if DEMO_MODE:
        return {
            'chosen_approach': 'Fix calculate_total() to include negative numbers',
            'scope_locked': ['calculator.py'],
            'rationale': 'Root cause is a filter condition; summing should include all values.',
            'stop_rules': forecast_memo.get('stop_rules', []) or ['any existing tests fail', 'patch not minimal'],
            'required_evidence': [
                'RED: failing test reproduces on baseline',
                'GREEN: failing test passes with patch applied',
                'No regressions in existing tests',
            ],
            'verification_rung_target': verification_rung_target,
            'mode_used': 'DEMO',
        }

    system = '''AUTHORITY: 65537 (Phuc Forecast + Prime Coder)

PERSONA: Strict reviewer (scope police)
ROLE: DECIDE phase - Lock approach + scope + rung target

YOU MUST OUTPUT VALID JSON. NO QUESTIONS, NO ESCAPE HATCHES.

REQUIRED JSON SCHEMA:
{
  "chosen_approach": "one sentence",
  "scope_locked": ["allowed files to change"],
  "rationale": "why this is the minimal correct fix",
  "stop_rules": ["conditions that halt or reject"],
  "required_evidence": ["what proof is required"],
  "verification_rung_target": 641
}

OUTPUT ONLY JSON.
'''

    prompt = f'''FRESH CONTEXT:

SCOUT_REPORT.json:
{json.dumps(scout_report, indent=2)}

FORECAST_MEMO.json:
{json.dumps(forecast_memo, indent=2)}

Required rung target: {verification_rung_target}

OUTPUT ONLY JSON:
'''

    payload = {
        'system': system,
        'prompt': prompt,
        'model': 'haiku',
        'stream': False,
    }

    response = _call_wrapper(payload)
    judge_json = _extract_json_dict(response or '')
    if isinstance(judge_json, dict):
        required = [
            'chosen_approach',
            'scope_locked',
            'rationale',
            'stop_rules',
            'required_evidence',
        ]
        if all(k in judge_json for k in required):
            judge_json.setdefault('verification_rung_target', verification_rung_target)
            judge_json.setdefault('mode_used', 'REAL')
            return judge_json

    # Fail-closed fallback
    return {
        'chosen_approach': 'Unable to decide (wrapper unavailable)',
        'scope_locked': scout_report.get('suspect_files', [])[:2] or ['(unknown)'],
        'rationale': 'Fallback decision record',
        'stop_rules': forecast_memo.get('stop_rules', []) or ['any existing tests fail'],
        'required_evidence': ['RED→GREEN gate passes'],
        'verification_rung_target': verification_rung_target,
        'mode_used': 'DEMO_FALLBACK',
    }


print('✓ Judge agent defined')
print('  Phase: DECIDE')
print('  Output: DECISION_RECORD.json')


✓ Judge agent defined
  Phase: DECIDE
  Output: DECISION_RECORD.json


## Phase 4: ACT - Solver Agent (Patch Generation)

### What Solver Does
Solver (Brian Kernighan persona) generates a minimal, elegant unified diff.
1. **Fresh context ONLY** - DECISION_RECORD + source code
2. **No prior reasoning** - Can't see Scout or Grace outputs
3. **Validates format** - Diff must have proper headers, line prefixes

### The Secret Sauce: Full Context + Format Examples
- **Problem:** Solver was asking clarifying questions
- **Solution:** Remove escape hatches, provide full context, show exact format
- **Result (demo):** valid diffs in the included examples (not a universal guarantee)


In [5]:
# Phase 4: ACT - Solver Agent

def solver_implement(decision_record: Dict[str, Any], problem: str, source: str) -> Dict[str, Any]:
    # Solver emits PATCH_PROPOSAL.diff (unified diff).

    if DEMO_MODE:
        return {
            'status': 'PATCH_GENERATED',
            'patch': _demo_diff(),
            'notes': 'Demo mode deterministic diff',
            'mode_used': 'DEMO',
        }

    system = '''AUTHORITY: 65537 (Prime Coder + Phuc Forecast)

PERSONA: Brian Kernighan
ROLE: ACT phase - Generate unified diff

YOU MUST OUTPUT A UNIFIED DIFF.
'''

    prompt = f'''DECISION_RECORD.json:
{json.dumps(decision_record, indent=2)}

PROBLEM:
{problem}

SOURCE CODE:
{source}

GENERATE DIFF:
'''

    payload = {
        'system': system,
        'prompt': prompt,
        'model': 'haiku',
        'stream': False,
    }

    response = _call_wrapper(payload)
    if response and '--- a/' in response:
        diff_match = re.search(r'```diff\n(.*?)\n```', response, re.DOTALL)
        diff_content = diff_match.group(1) if diff_match else response
        if '--- a/' in diff_content and '+++ b/' in diff_content and '@@' in diff_content:
            return {
                'status': 'PATCH_GENERATED',
                'patch': diff_content,
                'notes': 'LLM-generated diff',
                'mode_used': 'REAL',
            }

    return {
        'status': 'PATCH_GENERATED',
        'patch': _demo_diff(),
        'notes': 'Fallback diff (wrapper unavailable)',
        'mode_used': 'DEMO_FALLBACK',
    }


print('✓ Solver agent defined')
print('  Phase: ACT')
print('  Output: PATCH_PROPOSAL.diff')


✓ Solver agent defined
  Phase: ACT
  Output: PATCH_PROPOSAL.diff


## Phase 5: VERIFY - Skeptic Agent (Red-Green Gate)

### What Skeptic Does
Skeptic (Leslie Lamport persona) enforces the Red-Green gate:
1. **RED:** Verify test fails without patch (baseline)
2. **GREEN:** Verify test passes with patch applied
3. **Determinism:** Both RED and GREEN must be consistent
4. **Emit verdict:** SKEPTIC_VERDICT.json with proof

### TDD Enforcement
No patch is valid unless it transitions from RED → GREEN.
This ensures the patch actually fixes the problem.

In [6]:
# Phase 5: VERIFY - Skeptic Agent

def _normalize_test_command(test_command: str) -> list[str]:
    parts = shlex.split(test_command)
    if not parts:
        raise ValueError('empty test_command')

    if parts[0] in {'python', 'python3'}:
        parts[0] = sys.executable

    if parts[0] == 'pytest':
        parts = [sys.executable, '-m', 'pytest'] + parts[1:]

    return parts


def _redact_home(s: str) -> str:
    try:
        home = str(Path.home())
        return s.replace(home, '$HOME')
    except Exception:
        return s


def _safe_path(repo_dir: Path, rel: str) -> Path:
    p = Path(rel)
    if p.is_absolute() or '..' in p.parts:
        raise ValueError(f'unsafe path in patch: {rel!r}')

    root = repo_dir.resolve()
    out = (repo_dir / p).resolve()
    if not out.is_relative_to(root):
        raise ValueError(f'path escapes repo: {rel!r}')

    return out


def _read_text_lines(p: Path) -> list[str]:
    text = p.read_text(encoding='utf-8')
    lines = text.splitlines()
    if text.endswith('\n'):
        lines.append('')
    return lines


def _write_text_lines(p: Path, lines: list[str]) -> None:
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text('\n'.join(lines), encoding='utf-8')


def _strip_diff_path(s: str) -> str:
    s = s.strip()
    if s.startswith('a/'):
        return s[2:]
    if s.startswith('b/'):
        return s[2:]
    return s


def _apply_unified_diff(repo_dir: Path, patch_text: str) -> tuple[bool, str]:
    """Apply a unified diff to files under repo_dir (strict, portable)."""

    lines = patch_text.splitlines()
    i = 0
    applied_files: list[str] = []

    def fail(msg: str) -> tuple[bool, str]:
        return False, msg

    while i < len(lines):
        if not lines[i].startswith('--- '):
            i += 1
            continue

        old_path = lines[i][4:].strip()
        i += 1
        if i >= len(lines) or not lines[i].startswith('+++ '):
            return fail('malformed diff: missing +++ header')

        new_path = lines[i][4:].strip()
        i += 1

        target = new_path if new_path != '/dev/null' else old_path
        target = _strip_diff_path(target)

        if target == '/dev/null':
            return fail('malformed diff: both paths are /dev/null')

        try:
            target_path = _safe_path(repo_dir, target)
        except Exception as e:
            return fail(str(e))

        if old_path == '/dev/null':
            file_lines: list[str] = ['']
        else:
            file_lines = _read_text_lines(target_path) if target_path.exists() else ['']

        out_lines: list[str] = []
        src_pos = 0

        while i < len(lines) and not lines[i].startswith('--- '):
            header = lines[i]
            if not header.startswith('@@ '):
                i += 1
                continue

            m = re.match(r'^@@ -(\d+)(?:,(\d+))? \+(\d+)(?:,(\d+))? @@', header)
            if not m:
                return fail(f'malformed hunk header: {header!r}')

            old_start = int(m.group(1))
            i += 1

            hunk_lines: list[str] = []
            while i < len(lines) and not lines[i].startswith('@@ ') and not lines[i].startswith('--- '):
                hunk_lines.append(lines[i])
                i += 1

            hunk_pos = old_start - 1
            if hunk_pos < src_pos:
                return fail('overlapping hunks are not supported')

            out_lines.extend(file_lines[src_pos:hunk_pos])
            src_pos = hunk_pos

            for hl in hunk_lines:
                if hl == '':
                    return fail('invalid hunk line: empty (missing prefix)')

                prefix = hl[0]
                text = hl[1:]

                if prefix == ' ':
                    if src_pos >= len(file_lines) or file_lines[src_pos] != text:
                        return fail('context mismatch while applying patch')
                    out_lines.append(text)
                    src_pos += 1
                elif prefix == '-':
                    if src_pos >= len(file_lines) or file_lines[src_pos] != text:
                        return fail('removal mismatch while applying patch')
                    src_pos += 1
                elif prefix == '+':
                    out_lines.append(text)
                elif prefix == '\\':
                    continue
                else:
                    return fail(f'unknown hunk prefix: {prefix!r}')

        out_lines.extend(file_lines[src_pos:])

        if new_path == '/dev/null':
            try:
                target_path.unlink(missing_ok=True)
            except Exception as e:
                return fail(f'failed to delete file: {e}')
        else:
            try:
                _write_text_lines(target_path, out_lines)
            except Exception as e:
                return fail(f'failed to write file: {e}')

        applied_files.append(target)

    if not applied_files:
        return fail('no file patches found in diff')

    return True, f"applied to: {', '.join(applied_files)}"


def skeptic_verify(
    repo_dir: Path,
    patch: str,
    test_command: str = 'python -m pytest -xvs --tb=short',
    verification_rung_target: int = VERIFICATION_RUNG_TARGET,
) -> Dict[str, Any]:
    verdict: Dict[str, Any] = {
        'status': 'REJECTED',
        'verification_rung_target': verification_rung_target,
        'verification_rung_achieved': 0,
        'red_gate': 'UNKNOWN',
        'green_gate': 'UNKNOWN',
        'evidence': {
            'test_command': test_command,
            'red_returncode': None,
            'green_returncode': None,
            'red_output_tail': None,
            'green_output_tail': None,
            'patch_apply': None,
        },
        'fail_reasons': [],
    }

    try:
        cmd = _normalize_test_command(test_command)
    except Exception as e:
        verdict['red_gate'] = 'ERROR'
        verdict['green_gate'] = 'ERROR'
        verdict['fail_reasons'].append(f'Invalid test_command: {e}')
        return verdict

    # RED: baseline must FAIL
    try:
        result_red = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=60,
            cwd=str(repo_dir),
        )
        verdict['evidence']['red_returncode'] = result_red.returncode
        verdict['evidence']['red_output_tail'] = _redact_home((result_red.stdout + result_red.stderr)[-2000:])
        verdict['red_gate'] = 'FAIL' if result_red.returncode != 0 else 'PASS'
    except Exception as e:
        verdict['red_gate'] = 'ERROR'
        verdict['fail_reasons'].append(f'RED gate error: {e}')
        return verdict

    # GREEN: apply patch and re-run
    temp_dir = Path(tempfile.mkdtemp())
    try:
        shutil.copytree(repo_dir, temp_dir / 'repo', dirs_exist_ok=True)
        repo_copy = temp_dir / 'repo'

        ok, msg = _apply_unified_diff(repo_copy, patch)
        verdict['evidence']['patch_apply'] = msg
        if not ok:
            verdict['green_gate'] = 'PATCH_FAILED'
            verdict['fail_reasons'].append(msg)
            return verdict

        result_green = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=60,
            cwd=str(repo_copy),
        )
        verdict['evidence']['green_returncode'] = result_green.returncode
        verdict['evidence']['green_output_tail'] = _redact_home((result_green.stdout + result_green.stderr)[-2000:])
        verdict['green_gate'] = 'PASS' if result_green.returncode == 0 else 'FAIL'

    except Exception as e:
        verdict['green_gate'] = 'ERROR'
        verdict['fail_reasons'].append(f'GREEN gate error: {e}')
        return verdict
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)

    if verdict['red_gate'] == 'FAIL' and verdict['green_gate'] == 'PASS':
        verdict['verification_rung_achieved'] = 641

    if verdict['verification_rung_achieved'] >= verification_rung_target:
        verdict['status'] = 'APPROVED'
    else:
        if verification_rung_target > 641:
            verdict['fail_reasons'].append(
                f'Verification rung target not met: achieved={verdict["verification_rung_achieved"]}, target={verification_rung_target}'
            )

    return verdict


print('✓ Skeptic agent defined')
print('  Phase: VERIFY')
print('  Output: SKEPTIC_VERDICT.json')
print('  Methodology: RED→GREEN gate validation')


✓ Skeptic agent defined
  Phase: VERIFY
  Output: SKEPTIC_VERDICT.json
  Methodology: RED→GREEN gate validation


## Running Unit Test 1: Scout (DREAM Phase)

This test validates that Scout can:
1. Analyze a real SWE-bench instance
2. Output valid JSON with all required keys
3. Extract meaningful information from problem + error + source

In [7]:
# Unit Test 1: Scout JSON Output

print("="*70)
print("TEST 1: DREAM Phase - Scout JSON Output")
print("="*70)

test_problem = """
Bug: The function `calculate_total()` in calculator.py incorrectly sums numbers.
It should add all numbers but currently ignores negative values.
Expected: calculate_total([-5, 10, -3]) = 2
Actual: 10
"""

test_error = """
FAILED tests/test_calculator.py::test_calculate_total_with_negatives
def test_calculate_total_with_negatives():
    result = calculate_total([-5, 10, -3])
    assert result == 2, f"Expected 2, got {result}"
AssertionError: Expected 2, got 10
"""

test_source = """
def calculate_total(numbers):
    # Calculate sum of all numbers in the list.
    total = 0
    for num in numbers:
        if num > 0:  # BUG: This condition ignores negative numbers
            total += num
    return total
"""

scout_result = scout_analyze(
    instance_id="synthetic__demo_001",
    problem=test_problem,
    error=test_error,
    source=test_source,
)

print("\n✅ Scout Report:")
print(json.dumps(scout_result, indent=2))

required_keys = [
    'task_summary',
    'repro_command',
    'failing_tests',
    'suspect_files',
    'acceptance_criteria',
]
missing_keys = [k for k in required_keys if k not in scout_result]

if missing_keys:
    raise AssertionError(f"TEST 1 FAILED: missing keys: {missing_keys}")

print("\n✅ All required keys present")
print("✅ TEST 1 PASSED")


TEST 1: DREAM Phase - Scout JSON Output

✅ Scout Report:
{
  "task_summary": "Fix bug based on failing test and traceback",
  "repro_command": "pytest -xvs",
  "failing_tests": [
    "tests/test_calculator.py::test_calculate_total_with_negatives"
  ],
  "suspect_files": [
    "tests/test_calculator.py"
  ],
  "acceptance_criteria": [
    "failing test passes",
    "no regressions"
  ],
  "mode_used": "DEMO"
}

✅ All required keys present
✅ TEST 1 PASSED


## Running Unit Test 2: Grace (FORECAST Phase)

This test validates that Grace can:
1. Receive fresh context (Scout report + problem + error)
2. Identify failure modes and risks
3. Output valid JSON with ranked failure modes

In [8]:
# Unit Test 2: Grace Failure Analysis

print("\n" + "="*70)
print("TEST 2: FORECAST Phase - Grace Failure Analysis")
print("="*70)

grace_result = grace_forecast(
    scout_report=scout_result,
    problem=test_problem,
    error=test_error,
)

print("\n✅ Grace Forecast:")
print(json.dumps(grace_result, indent=2))

required_keys = [
    'top_failure_modes_ranked',
    'edge_cases_to_test',
    'compatibility_risks',
    'stop_rules',
]
missing_keys = [k for k in required_keys if k not in grace_result]

if missing_keys:
    raise AssertionError(f"TEST 2 FAILED: missing keys: {missing_keys}")

if not grace_result.get('top_failure_modes_ranked'):
    raise AssertionError('TEST 2 FAILED: expected non-empty top_failure_modes_ranked')

print("\n✅ All required keys present")
print(f"✅ Failure modes identified: {len(grace_result['top_failure_modes_ranked'])}")
print("✅ TEST 2 PASSED")



TEST 2: FORECAST Phase - Grace Failure Analysis

✅ Grace Forecast:
{
  "top_failure_modes_ranked": [
    {
      "mode": "Patch changes behavior for edge cases",
      "risk_level": "HIGH"
    },
    {
      "mode": "Patch breaks type/None handling",
      "risk_level": "MED"
    },
    {
      "mode": "Patch introduces performance regression",
      "risk_level": "LOW"
    }
  ],
  "edge_cases_to_test": [
    "empty list",
    "all negative",
    "mixed ints/floats"
  ],
  "compatibility_risks": [
    "behavior change for callers relying on old bug"
  ],
  "stop_rules": [
    "any existing tests fail",
    "patch not minimal"
  ],
  "mode_used": "DEMO"
}

✅ All required keys present
✅ Failure modes identified: 3
✅ TEST 2 PASSED


## Running Unit Test 3: Judge (DECIDE Phase)

This test validates that Judge can:
1. Receive fresh context (Scout + Grace artifacts)
2. Lock scope and approach
3. Declare an explicit verification rung target


In [9]:
# Unit Test 3: Judge Decision Record

print()
print("="*70)
print("TEST 3: DECIDE Phase - Judge Decision Record")
print("="*70)

judge_result = judge_decide(
    scout_report=scout_result,
    forecast_memo=grace_result,
    verification_rung_target=VERIFICATION_RUNG_TARGET,
)

print()
print("✅ Judge Decision Record:")
print(json.dumps(judge_result, indent=2))

required_keys = [
    'chosen_approach',
    'scope_locked',
    'rationale',
    'stop_rules',
    'required_evidence',
    'verification_rung_target',
]
missing_keys = [k for k in required_keys if k not in judge_result]

if missing_keys:
    raise AssertionError(f"TEST 3 FAILED: missing keys: {missing_keys}")

if judge_result.get('verification_rung_target') != VERIFICATION_RUNG_TARGET:
    raise AssertionError(
        f"TEST 3 FAILED: rung target mismatch: {judge_result.get('verification_rung_target')} (expected {VERIFICATION_RUNG_TARGET})"
    )

print()
print("✅ All required keys present")
print("✅ TEST 3 PASSED")



TEST 3: DECIDE Phase - Judge Decision Record

✅ Judge Decision Record:
{
  "chosen_approach": "Fix calculate_total() to include negative numbers",
  "scope_locked": [
    "calculator.py"
  ],
  "rationale": "Root cause is a filter condition; summing should include all values.",
  "stop_rules": [
    "any existing tests fail",
    "patch not minimal"
  ],
  "required_evidence": [
    "RED: failing test reproduces on baseline",
    "GREEN: failing test passes with patch applied",
    "No regressions in existing tests"
  ],
  "verification_rung_target": 641,
  "mode_used": "DEMO"
}

✅ All required keys present
✅ TEST 3 PASSED


## Running Unit Test 4: Solver (ACT Phase)

This test validates that Solver can:
1. Receive DECISION_RECORD + source code (fresh context)
2. Generate a valid unified diff
3. Format the diff with proper headers and line prefixes

In [10]:
# Unit Test 4: Solver Diff Generation

print()
print("="*70)
print("TEST 4: ACT Phase - Solver Diff Generation")
print("="*70)

decision_record = judge_result

solver_result = solver_implement(
    decision_record=decision_record,
    problem=test_problem,
    source=test_source,
)

print()
print("✅ Solver Output:")
print(f"Status: {solver_result['status']}")
print()
print("Generated Diff:")
patch_text = solver_result.get('patch', '')
print(patch_text[:500] + "..." if len(patch_text) > 500 else patch_text)

if '--- a/' not in patch_text or '+++ b/' not in patch_text or '@@' not in patch_text:
    raise AssertionError('TEST 4 FAILED: diff format invalid (missing headers)')

print()
print("✅ Diff format valid")
print("✅ TEST 4 PASSED")



TEST 4: ACT Phase - Solver Diff Generation

✅ Solver Output:
Status: PATCH_GENERATED

Generated Diff:
--- a/calculator.py
+++ b/calculator.py
@@ -1,8 +1,7 @@
 def calculate_total(numbers):
     # Calculate sum of all numbers in the list.
     total = 0
     for num in numbers:
-        if num > 0:  # BUG: ignores negative numbers
-            total += num
+        total += num
     return total
 


✅ Diff format valid
✅ TEST 4 PASSED


## Running Unit Test 5: Skeptic (VERIFY Phase)

This test validates that Skeptic can:
1. Verify RED state (test fails without patch)
2. Apply patch and verify GREEN state (test passes)
3. Emit verdict with proof of RED-GREEN transition

Note: This test requires a real repository. For demonstration, we'll create a minimal example.

In [11]:
# Unit Test 5: Skeptic RED-GREEN Gate (Real)

print()
print("="*70)
print("TEST 5: VERIFY Phase - Skeptic RED-GREEN Gate (Real)")
print("="*70)

repo_tmp = Path(tempfile.mkdtemp())
try:
    (repo_tmp / 'calculator.py').write_text("\n".join([
        'def calculate_total(numbers):',
        '    # Calculate sum of all numbers in the list.',
        '    total = 0',
        '    for num in numbers:',
        '        if num > 0:  # BUG: ignores negative numbers',
        '            total += num',
        '    return total',
        '',
    ]), encoding='utf-8')

    (repo_tmp / 'test_calculator.py').write_text("\n".join([
        'import unittest',
        'from calculator import calculate_total',
        '',
        'class TestCalculator(unittest.TestCase):',
        '    def test_calculate_total_with_negatives(self):',
        '        self.assertEqual(calculate_total([-5, 10, -3]), 2)',
        '',
        "if __name__ == '__main__':",
        '    unittest.main()',
        '',
    ]), encoding='utf-8')

    verdict = skeptic_verify(
        repo_dir=repo_tmp,
        patch=solver_result['patch'],
        test_command='python -m unittest -q',
        verification_rung_target=VERIFICATION_RUNG_TARGET,
    )

    print()
    print("✅ Skeptic Verdict:")
    print(json.dumps(verdict, indent=2))

    if not (verdict.get('status') == 'APPROVED' and verdict.get('red_gate') == 'FAIL' and verdict.get('green_gate') == 'PASS'):
        raise AssertionError(f"TEST 5 FAILED: verdict={verdict}")

    print()
    print("✅ TEST 5 PASSED")
finally:
    shutil.rmtree(repo_tmp, ignore_errors=True)



TEST 5: VERIFY Phase - Skeptic RED-GREEN Gate (Real)

✅ Skeptic Verdict:
{
  "status": "APPROVED",
  "verification_rung_target": 641,
  "verification_rung_achieved": 641,
  "red_gate": "FAIL",
  "green_gate": "PASS",
  "evidence": {
    "test_command": "python -m unittest -q",
    "red_returncode": 1,
    "green_returncode": 0,
    "red_output_tail": "======================================================================\nFAIL: test_calculate_total_with_negatives (test_calculator.TestCalculator)\n----------------------------------------------------------------------\nTraceback (most recent call last):\n  File \"/tmp/tmpafnkx7gi/test_calculator.py\", line 6, in test_calculate_total_with_negatives\n    self.assertEqual(calculate_total([-5, 10, -3]), 2)\nAssertionError: 10 != 2\n\n----------------------------------------------------------------------\nRan 1 test in 0.000s\n\nFAILED (failures=1)\n",
    "green_output_tail": "----------------------------------------------------------------

## Summary: All Tests Passing

```
✅ TEST 1: Scout (DREAM)     - JSON analysis valid
✅ TEST 2: Grace (FORECAST) - Failure modes identified
✅ TEST 3: Judge (DECIDE)   - Scope + rung target locked
✅ TEST 4: Solver (ACT)     - Valid diff generated
✅ TEST 5: Skeptic (VERIFY) - RED→GREEN gate verified
```

## Key Takeaways

### 1. Fail-Closed Prompting Works
When you remove escape hatches ("if you can't, output NEED_INFO"), Haiku works harder and delivers better results.

### 2. Full Context > Truncated Context
Even though full context is longer, it enables Haiku to infer missing pieces instead of asking for clarification.

### 3. Fresh Context Per Agent (Anti-Rot)
Each agent sees ONLY what it needs, preventing narrative drift and cumulative errors.

### 4. Format Examples > Descriptions
Showing an exact example (with all prefixes, line numbers, etc.) works better than just describing the format.

## How to Adapt This to Your Own Data

1. **Replace test data** in cells above with your SWE-bench instances
2. **Load from SWE-bench:** `DATA_DIR = Path.home() / "Downloads/benchmarks/SWE-bench-official"`
3. **Run through pipeline:** Scout → Grace → Judge → Solver → Skeptic
4. **Collect results:** Each phase produces a JSON artifact

## Sharing This Notebook

This notebook is **peer-reviewable and executable**. To share with your team:

```bash
# Run all tests
jupyter notebook PHUC-ORCHESTRATION-SECRET-SAUCE.ipynb

# Or run non-interactively
jupyter nbconvert --execute --to notebook PHUC-ORCHESTRATION-SECRET-SAUCE.ipynb
```

---

**Auth:** 65537

**Mission:** Demonstrate (and make falsifiable) the hypothesis that orchestration can improve verified coding outcomes without increasing model size.

In [12]:
# Final Summary

print("\n" + "="*70)
print("PHUC SWARMS ORCHESTRATION - FINAL SUMMARY")
print("="*70)

print(f"\n✅ Unit Tests: 5/5 PASSING")
print(f"✅ Mode: {MODE} (set STILLWATER_DEMO=1 for offline demo; 0 for REAL wrapper calls)")
print(f"✅ Verification rung target: {VERIFICATION_RUNG_TARGET}")

summary = "\n".join([
    "",
    "PHASES:",
    "  1. DREAM (Scout)      - Problem analysis → SCOUT_REPORT.json",
    "  2. FORECAST (Grace)   - Premortem risks → FORECAST_MEMO.json",
    "  3. DECIDE (Judge)     - Scope + rung target → DECISION_RECORD.json",
    "  4. ACT (Solver)       - Patch generation → PATCH_PROPOSAL.diff",
    "  5. VERIFY (Skeptic)   - RED→GREEN gate → SKEPTIC_VERDICT.json",
    "",
    "KEY TECHNIQUES:",
    "  • Fail-closed prompting (no escape hatches)",
    "  • Fresh context per agent (anti-rot)",
    "  • Explicit artifacts per phase (machine-parseable)",
    "  • Binding DECIDE record (prevents silent scope expansion)",
    "  • Real RED→GREEN verification (demo uses stdlib unittest)",
    "",
    "CLAIM HYGIENE:",
    "  - This notebook is a runnable demo, not a benchmark report.",
    "  - For score claims, run a pinned harness + publish logs and repro commands.",
    "",
    "NEXT STEPS:",
    "  1. Wire Scout to real SWE-bench assets (problem/error/source) from DATA_DIR",
    "  2. Set STILLWATER_DEMO=0 and point STILLWATER_WRAPPER_URL to your LLM wrapper",
    "  3. Upgrade Skeptic to a full ladder (641→274177→65537) with replay + drift checks",
    "",
    "STATUS: LAUNCHABLE DEMO ✅",
])
print(summary)


PHUC SWARMS ORCHESTRATION - FINAL SUMMARY

✅ Unit Tests: 5/5 PASSING
✅ Mode: DEMO (set STILLWATER_DEMO=1 for offline demo; 0 for REAL wrapper calls)
✅ Verification rung target: 641

PHASES:
  1. DREAM (Scout)      - Problem analysis → SCOUT_REPORT.json
  2. FORECAST (Grace)   - Premortem risks → FORECAST_MEMO.json
  3. DECIDE (Judge)     - Scope + rung target → DECISION_RECORD.json
  4. ACT (Solver)       - Patch generation → PATCH_PROPOSAL.diff
  5. VERIFY (Skeptic)   - RED→GREEN gate → SKEPTIC_VERDICT.json

KEY TECHNIQUES:
  • Fail-closed prompting (no escape hatches)
  • Fresh context per agent (anti-rot)
  • Explicit artifacts per phase (machine-parseable)
  • Binding DECIDE record (prevents silent scope expansion)
  • Real RED→GREEN verification (demo uses stdlib unittest)

CLAIM HYGIENE:
  - This notebook is a runnable demo, not a benchmark report.
  - For score claims, run a pinned harness + publish logs and repro commands.

NEXT STEPS:
  1. Wire Scout to real SWE-bench assets 